In [1]:
# 📦 INSTALLATION
!pip install flashtext openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=779120be5cec5807ba707d1bf5753591c33098c37fc2798b97a5f653e548983e
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext


In [4]:

# 📚 IMPORTS
import pandas as pd
import os
import zipfile
from google.colab import files
from flashtext import KeywordProcessor
from tqdm import tqdm
import concurrent.futures
import shutil
import re

def main():
    # 📥 TÉLÉVERSEMENT DES FICHIERS
    print("➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx...")
    uploaded = files.upload()
    zip_filename = list(uploaded.keys())[0]

    print("➡️ Téléversez le fichier Excel des taxons (colonnes: Virus, Variant, Genus, Family)...")
    uploaded = files.upload()
    taxon_filename = list(uploaded.keys())[0]

    # 📂 EXTRACTION DU ZIP
    input_dir = "extracted_files"
    os.makedirs(input_dir, exist_ok=True)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(input_dir)

    # 🔍 CHARGEMENT DES FICHIERS EXCEL
    excel_files = [
        os.path.join(root, file)
        for root, _, files_ in os.walk(input_dir)
        for file in files_
        if file.lower().endswith(".xlsx")
    ]
    if not excel_files:
        raise Exception("❌ Aucun fichier .xlsx trouvé dans le dossier.")

    # 📄 CHARGEMENT DES DONNÉES TAXONOMIQUES
    df_taxons = pd.read_excel(taxon_filename, engine='openpyxl')
    df_taxons.columns = df_taxons.columns.str.strip()

    required_columns = {'Virus', 'Variant', 'Genus', 'Family'}
    if not required_columns.issubset(df_taxons.columns):
        missing = required_columns - set(df_taxons.columns)
        raise ValueError(f"❌ Colonnes manquantes dans le fichier taxons: {missing}")

    # 🧠 CRÉATION DES DICTIONNAIRES DE CORRESPONDANCE
    dict_virus_variants = {}
    dict_virus_genus = {}
    dict_genus_family = {}

    for row in df_taxons.itertuples(index=False):
        virus = str(row.Virus).strip()
        variants = [v.strip() for v in str(row.Variant).replace(";", ",").split(",") if v.strip()]
        dict_virus_variants[virus] = variants

        genus = str(row.Genus).strip()
        if genus and genus.lower() != 'nan':
            dict_virus_genus[virus] = genus

        family = str(row.Family).strip()
        if genus and genus.lower() != 'nan' and family and family.lower() != 'nan':
            dict_genus_family[genus] = family

    # 🔍 INITIALISATION DES PROCESSEURS FLASHTEXT
    kp_virus = KeywordProcessor(case_sensitive=False)
    for virus, variants in dict_virus_variants.items():
        for variant in variants:
            kp_virus.add_keyword(variant, virus)
        kp_virus.add_keyword(virus, virus)

    kp_genus = KeywordProcessor(case_sensitive=False)
    for virus, genus in dict_virus_genus.items():
        kp_genus.add_keyword(virus, genus)
        kp_genus.add_keyword(genus, genus)

    kp_family = KeywordProcessor(case_sensitive=False)
    for genus, family in dict_genus_family.items():
        kp_family.add_keyword(genus, family)

    # ⚙️ FONCTIONS DE DÉTECTION
    def detecter_virus(texte):
        if pd.isna(texte):
            return set()
        found = kp_virus.extract_keywords(str(texte))
        return set(found) if found else set()

    def detecter_genus(texte):
        if pd.isna(texte):
            return set()
        found = kp_genus.extract_keywords(str(texte))
        return set(found) if found else set()

    def detecter_famille(texte):
        if pd.isna(texte):
            return set()
        found = kp_family.extract_keywords(str(texte))
        return set(found) if found else set()

    # ⚙️ FONCTION DE TRAITEMENT D'UN FICHIER
    def traiter_fichier(filepath):
        try:
            filename = os.path.basename(filepath)
            print(f"🔎 Traitement de : {filename}")

            df = pd.read_excel(filepath, engine='openpyxl')
            colonnes_requises = {"Titre", "Résumé", "Entities_bc5cdr", "Entities_bionlp"}

            if not colonnes_requises.issubset(df.columns):
                missing = colonnes_requises - set(df.columns)
                print(f"❌ Colonnes manquantes dans {filename}: {missing}")
                return None

            titre_resume = df["Titre"].fillna("") + " " + df["Résumé"].fillna("")
            entites_concat = df["Entities_bionlp"].fillna("") + " " + df["Entities_bc5cdr"].fillna("")

            virus_detectés_liste = []
            genus_par_défaut_liste = []

            for t1, t2 in zip(titre_resume, entites_concat):
                virus_set = detecter_virus(t1) | detecter_virus(t2)
                if virus_set:
                    virus_detectés_liste.append(", ".join(sorted(virus_set)))
                    genus_par_défaut_liste.append("")
                else:
                    virus_detectés_liste.append("")
                    genus_alt = detecter_genus(t1)
                    genus_par_défaut_liste.append(", ".join(sorted(genus_alt)) if genus_alt else "")

            df["Virus_detectés"] = virus_detectés_liste
            df["Genus_par_défaut"] = genus_par_défaut_liste

            df["Genus_detectés"] = df["Virus_detectés"].apply(
                lambda viruses: ", ".join(sorted({
                    dict_virus_genus.get(virus.strip(), "")
                    for virus in viruses.split(",")
                    if virus.strip() in dict_virus_genus
                })) if viruses.strip() else ""
            )

            df["Genus_detectés"] = df.apply(
                lambda row: row["Genus_detectés"] if row["Genus_detectés"] else row["Genus_par_défaut"],
                axis=1
            )

            df["Familles_detectées"] = df["Genus_detectés"].apply(
                lambda genera: ", ".join(sorted({
                    dict_genus_family.get(genus.strip(), "")
                    for genus in genera.split(",")
                    if genus.strip() in dict_genus_family
                })) if pd.notna(genera) else ""
            )

            # Fallback pour présence virus par regex si aucune détection précédente
            df["presence_virus"] = [
                ", ".join(re.findall(r'\b[\w\-]*(virus|dae)\b', txt, re.IGNORECASE))
                if (not v or v.strip() == "") and (not g or g.strip() == "") and re.search(r'\b[\w\-]*(virus|dae)\b', txt, re.IGNORECASE) else ""
                for v, g, txt in zip(df["Virus_detectés"], df["Genus_detectés"], titre_resume)

                ]


            return df, filename
        except Exception as e:
            print(f"❌ Erreur lors du traitement de {filepath}: {str(e)}")
            return None

    # 📂 DOSSIER DE SORTIE
    output_dir = "fichiers_des_virus"
    os.makedirs(output_dir, exist_ok=True)

    # 🔁 TRAITEMENT DES FICHIERS EN PARALLÈLE
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(traiter_fichier, filepath): filepath for filepath in excel_files}

        for future in tqdm(concurrent.futures.as_completed(futures),
                          total=len(futures),
                          desc="🔄 Traitement des fichiers"):
            result = future.result()
            if result is not None:
                df, filename = result
                output_path = os.path.join(output_dir, filename)
                df.to_excel(output_path, index=False, engine='openpyxl')
                print(f"✅ Sauvegardé : {output_path}")

    # 📦 COMPRESSION EN FICHIER ZIP
    zip_output_path = os.path.join(os.getcwd(), "fichiers_virus_annotés")
    shutil.make_archive(zip_output_path, "zip", output_dir)

    # 📤 TÉLÉCHARGEMENT DU ZIP
    files.download(f"{zip_output_path}.zip")
    print(f"📦 Archive ZIP générée et prête à être téléchargée: {zip_output_path}.zip")

if __name__ == "__main__":
    main()


➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx...


Saving Step3_extractions_des_pays_dans_les_affiliations.zip to Step3_extractions_des_pays_dans_les_affiliations (1).zip
➡️ Téléversez le fichier Excel des taxons (colonnes: Virus, Variant, Genus, Family)...


Saving liste_virus_final.xlsx to liste_virus_final (1).xlsx
🔎 Traitement de : Ivory Coast_entites.xlsx🔎 Traitement de : Sudan_entites.xlsx

🔎 Traitement de : Mozambique_entites.xlsx
🔎 Traitement de : Seychelles_entites.xlsx
🔎 Traitement de : Tanzania_entites.xlsx
🔎 Traitement de : Madagascar_entites.xlsx


🔄 Traitement des fichiers:   0%|          | 0/54 [00:00<?, ?it/s]

🔎 Traitement de : Mali_entites.xlsx


...

🔄 Traitement des fichiers: 100%|██████████| 54/54 [01:56<00:00,  2.16s/it]

✅ Sauvegardé : fichiers_des_virus/South Africa1_entites.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📦 Archive ZIP générée et prête à être téléchargée: /content/fichiers_virus_annotés.zip
